# Creando un pipeline en Step Functions

Ahora que tenemos nuestro proyecto de ciencia de datos construido, queremos implementarlo de una manera sólida y repetible. Para esto, implementaremos el ETL usando AWS Glue, y luego entrenaremos y transformaremos por lotes la entrada usando la integración de SageMaker con Amazon Step Functions.

Este cuaderno lo guiará a través de este proceso paso a paso.

Pero primero debe crear o configurar su propio cubo. El SDK de SageMaker es una buena forma de empezar.

## 1. Subiendo archivos al bucket S3 external

In [2]:
import boto3
import os

In [3]:
account_id = boto3.client("sts").get_caller_identity()["Account"]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [5]:
bucket = "fashionstore-datalake-external-" + str(account_id)

In [6]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join('data','billing', 'billing_sm.csv')).upload_file('../data/billing_sm.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join('data','reseller', 'reseller_sm.csv')).upload_file('../data/reseller_sm.csv')

## 2. Ejecutando un Crawler en Glue (creará 2 tablas en el esquema fashion_external)

Para usar esta información en csv en el contexto de un ETL en Glue, primero se deben crear las tablas en Glue Data Catalog, para ello se crea un crawler en Glue que apunte a la carpeta data del bucket external en S3. El crawler intentará averiguar los tipos de datos de cada columna. 

Accedemos a Glue a la opción Crawler.
Clic el enlace <a href='https://us-east-1.console.aws.amazon.com/glue/home?region=us-east-1#/v2/data-catalog/crawlers'> Link </a>       

Seleccionar el crawler : crawler_data_external_fashion y clic en Run.

<img src='img/c202.png' style='width:500px' />

Después de un minuto, se deben haber creado 2 tablas en el esquema fashion_external.

<img src='img/c10.png' style='width:500px' />

## 4. Create Glue Job

Para crear un job en Glue, clic en el siguiente enlace.

 <a href='https://us-east-1.console.aws.amazon.com/gluestudio/home?region=us-east-1#/jobs'> Link</a> 


Seleccionar : Spark script editor y clic en Create.

<img src='img/c111.png' style='width:700px' />

Asignamos el nombre al job : ETL_PIPELINE


Y pegamos el contenido el código : ETL_PIPELINE.py

<img src='img/c15.png' style='width:700px' />

Clic en Job details y seleccionamos el rol : RoleGlue-fashionstore, y clic en Save.

<img src='img/c13.png' style='width:700px' />    

## 3. Crear un nuevo state machine en Step Functions

In [7]:
from sagemaker import get_execution_role
your_role = get_execution_role()

In [8]:
definition = open('step_function.json', 'r').read().replace('your_role', your_role).replace('account_id', account_id)
print(definition)

{
  "Comment": "ML Pipeline",
  "StartAt": "Start Glue Job",
  "States": {
    "Start Glue Job": {
      "Type": "Task",
      "Resource": "arn:aws:states:::glue:startJobRun.sync",
      "Parameters": {
        "JobName": "ETL_PIPELINE"
      },
      "Next": "Train model (XGBoost)"
    },
    "Train model (XGBoost)": {
      "Resource": "arn:aws:states:::sagemaker:createTrainingJob.sync",
      "Parameters": {
        "AlgorithmSpecification": {
          "TrainingImage": "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest",
          "TrainingInputMode": "File"
        },
        "OutputDataConfig": {
          "S3OutputPath": "s3://fashionstore-datalake-sm-971489366207/models"
        },
        "StoppingCondition": {
          "MaxRuntimeInSeconds": 86400
        },
        "ResourceConfig": {
          "InstanceCount": 1,
          "InstanceType": "ml.m4.xlarge",
          "VolumeSizeInGB": 30
        },
        "RoleArn": "arn:aws:iam::971489366207:role/StackFashionDataL

Ir al servicio de Step functions y crear un nuevo state machine.

Clic el enlace <a href='https://us-east-1.console.aws.amazon.com/states/home?region=us-east-1#/statemachines'> Link </a> 

Clic en Create state machine

Seleccionar : Write your workflow in code
Y pegamos el json en el bloque Definition.

Clic en Next.

<img src='img/c100.png' style='width:500px' />

Asignamos un nombre : **sf-external-fashionstore-ml**

Elegir el rol : **RoleSM-fashionstore-datalake**.


Ejecutamos el state machine creado, clic en Start execution.

<img src='img/c21.png' style='width:500px' />

Y se empieza a ejecutar el pipeline.

<img src='img/c58.png' style='width:500px' />

Si nos vamos a Glue a la opción Jobs, vemos que se ha iniciado la ejecución.

<img src='img/c102.png' style='width:500px' />

Consulte el training job y el modelo que se creó en Sagemaker.

Sagemaker training job : <a href='https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs'> Link </a> 

Modelo creado en Sagemaker : <a href='https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/models'> Link </a>